<a href="https://colab.research.google.com/github/pranavsrinivas29/Knowledge-Graph/blob/main/SPARQL_Wikidata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Querying Wikidata with mkwikidata ###

[mkwikidata](https://pypi.org/project/mkwikidata/#description) is a package for running SPARQL queries on Wikidata

We have to install the following packages:

**mkwikidata**

In [1]:
!pip install mkwikidata

In [2]:
import mkwikidata, pprint
import pandas as pd

#### **Query 1**:  Show all narrated cities of books and literary works and their geo coordination in Wikidata

Following Wikidata classes/instances and properties are required:

[wd:Q571](https://www.wikidata.org/wiki/Q571), [wd:Q7725634](https://www.wikidata.org/wiki/Q7725634), [wd:Q515](https://www.wikidata.org/wiki/Q515),  [wdt:P31](https://www.wikidata.org/wiki/Property:P31), [wdt:P840](https://www.wikidata.org/wiki/Property:P840), [wdt:P625](https://www.wikidata.org/wiki/Property:P625) , [wdt:P279](https://www.wikidata.org/wiki/Property:P279)

In [4]:
query = """
SELECT ?label ?placelabel ?coord
WHERE
{
  VALUES ?type {wd:Q571 wd:Q7725634} #book or literary work
  ?item wdt:P31 ?type . #of type ?type
  ?item rdfs:label ?label FILTER (lang(?label) = "en") .
  ?item wdt:P840 ?place . #narration place ?place
  ?place wdt:P31/wdt:P279* wd:Q515 . # place is an instance or a subclass of city
  ?place wdt:P625 ?coord . #has coordinations ?ccord
  ?place rdfs:label ?placelabel FILTER (lang(?placelabel) = "en") .
}
"""
query_result = mkwikidata.run_query(query, params={ })
query_result

{'head': {'vars': ['label', 'placelabel', 'coord']},
 'results': {'bindings': [{'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(-122.416388888 37.7775)'},
    'label': {'xml:lang': 'en',
     'type': 'literal',
     'value': "All Tomorrow's Parties"},
    'placelabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'San Francisco'}},
   {'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(-122.416388888 37.7775)'},
    'label': {'xml:lang': 'en',
     'type': 'literal',
     'value': "The Kitchen God's Wife"},
    'placelabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'San Francisco'}},
   {'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(-122.416388888 37.7775)'},
    'label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Zodiac'},
    'placelabel': {'xml:

The json_normalize() function is used to convert the JSON string into a DataFrame for a more (human)readable output

In [7]:
results_df = pd.json_normalize(query_result['results']['bindings'])
results_df[['label.value', 'placelabel.value', 'coord.value']]

,label.value,placelabel.value,coord.value
0,All Tomorrow's Parties,San Francisco,Point(-122.416388888 37.7775)
1,The Kitchen God's Wife,San Francisco,Point(-122.416388888 37.7775)
2,Zodiac,San Francisco,Point(-122.416388888 37.7775)
3,American Gods,San Francisco,Point(-122.416388888 37.7775)
4,Johnny Kellock Died Today,Halifax,Point(-63.571388888 44.647777777)
...,...,...,...
11258,Buddenbrooks,Lübeck,Point(10.686388888 53.869722222)
11259,Martin Paz,Lima,Point(-77.0375 -12.06)
11260,The Life to Come,Colombo,Point(79.833333333 6.916666666)
11261,The Seven Moons of Maali Almeida,Colombo,Point(79.833333333 6.916666666)


Let's [visualize the query](https://query.wikidata.org/#%23defaultView%3AMap%0ASELECT%20%3Flabel%20%3Fplacelabel%20%3Fcoord%0AWHERE%0A%7B%0A%20%20VALUES%20%3Ftype%20%7Bwd%3AQ571%20wd%3AQ7725634%7D%20%20%23%20book%20or%20literary%20work%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%20.%0A%20%20%3Fitem%20rdfs%3Alabel%20%3Flabel%20FILTER%20%28lang%28%3Flabel%29%20%3D%20%22en%22%29%20.%0A%20%20%3Fitem%20wdt%3AP840%20%3Fplace%20.%20%20%23%20narration%20place%20is%20%3Fplace%0A%20%20%3Fplace%20wdt%3AP625%20%3Fcoord%20.%0A%20%20%3Fplace%20rdfs%3Alabel%20%3Fplacelabel%20FILTER%20%28lang%28%3Fplacelabel%29%20%3D%20%22en%22%29%20.%0A%7D%20%0A) result in Wikidata Query Service\!

#### **Query 2**: What is the most common narration country in Wikidata books and literature works? ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P17](https://www.wikidata.org/wiki/Property:P17)

In [10]:
query = """
SELECT ?countrylabel (COUNT(?country) AS ?count)
WHERE
{
  VALUES ?type {wd:Q571 wd:Q7725634}  # book or literary work
  ?item wdt:P31 ?type .
  ?item wdt:P840 ?place . # narration place is ?place
  ?place wdt:P17 ?country .
  ?country rdfs:label ?countrylabel FILTER (lang(?countrylabel) = "en") .

}
  GROUP BY ?countrylabel
  ORDER BY DESC(?count)

"""
query_result = mkwikidata.run_query(query, params={ })
results_df = pd.json_normalize(query_result['results']['bindings'])
results_df[['countrylabel.value', 'count.value']]

,countrylabel.value,count.value
0,United States of America,2553
1,United Kingdom,1605
2,France,1355
3,Italy,742
4,Spain,616
...,...,...
361,Three Kingdoms of Korea,1
362,Goryeo,1
363,Later Three Kingdoms,1
364,Later Silla,1


Let's try out the bubble chart visualization of [the query result](https://query.wikidata.org/#%23defaultView%3ABubbleChart%0ASELECT%20%3Fcountrylabel%20%28COUNT%28%3Fcountry%29%20AS%20%3Fcount%29%0AWHERE%0A%7B%0A%20%20VALUES%20%3Ftype%20%7Bwd%3AQ571%20wd%3AQ7725634%7D%20%20%23%20book%20or%20literary%20work%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%20.%0A%20%20%3Fitem%20wdt%3AP840%20%3Fplace%20.%20%23%20narration%20place%20is%20%3Fplace%0A%20%20%3Fplace%20wdt%3AP17%20%3Fcountry%20.%0A%20%20%3Fcountry%20rdfs%3Alabel%20%3Fcountrylabel%20FILTER%20%28lang%28%3Fcountrylabel%29%20%3D%20%22en%22%29%20.%0A%0A%7D%20%0A%20%20GROUP%20BY%20%3Fcountrylabel%0A%20%20ORDER%20BY%20DESC%28%3Fcount%29%0A) in Wikidata!

### Querying Wikidata with SPARQLWrapper ###

Alternatively, Wikidata KG and other KGs can be queried using the [SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/main.html) – a python interface for any SPARQL Endpoint.

For this, we have to install and import the following packages:


**SPARQLWrapper**

In [11]:
!pip install -q sparqlwrapper    #install SPARQLwrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF

In [13]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql") #determine SPARQL endpoint
sparql.setReturnFormat(JSON) #determine the output format

#### **Query 3**: Being a writer is dangerous! Let's see what is the most common cause of death among writers? ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P106](https://www.wikidata.org/wiki/Property:P106), [wdt:509](https://www.wikidata.org/wiki/Property:P509), [wd:Q36180](https://www.wikidata.org/wiki/Q36180)


In [16]:
# Add User-Agent header
sparql.addCustomHttpHeader('User-Agent', 'MyBot/1.0 (myemail@example.com)')
#Replace 'MyBot/1.0' and '[redacted link]' with appropriate information.

In [18]:
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE
{
  ?person wdt:P106 wd:Q36180 . #occupation writer
  ?person wdt:P509 ?cause . #?cause is a death cause
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)
 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.json_normalize(results['results']['bindings'])
results_df[['causeLabel.value', 'count.value']]

,causeLabel.value,count.value
0,cancer,1147
1,myocardial infarction,1100
2,tuberculosis,580
3,pneumonia,549
4,stroke,443
...,...,...
290,death from scuba diving,2
291,November 2015 Paris attacks,2
292,CaVe,2
293,Q78698047,2


#### **Query 4**: And if we talk about writers that died before 1800? ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P570](https://www.wikidata.org/wiki/Property:P570)

In [19]:
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE
{
  ?person wdt:P106 wd:Q36180; #ocupation writer
          wdt:P509 ?cause ; #?cause is a death cause
          wdt:P570 ?date . #death date
  FILTER (?date < "1800-01-01T00:00:00Z"^^xsd:dateTime) .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)

 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.json_normalize(results['results']['bindings'])
results_df[['causeLabel.value', 'count.value']]

,causeLabel.value,count.value
0,decapitation,89
1,plague,64
2,disease,55
3,tuberculosis,45
4,death by burning,35
5,drowning,31
6,hanging,26
7,smallpox,23
8,stroke,21
9,killed in action,18


So, being a writer before 1800 could often leave you without the head. :)


#### **Query 5**: Now let's see how writers tend to kill their characters. ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P674](https://www.wikidata.org/wiki/Property:P674)

In [20]:
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?character) AS ?count)

WHERE
{
  VALUES ?type {wd:Q571 wd:Q7725634}  # book or literary work
  ?item wdt:P31 ?type .
  ?item wdt:P674 ?character .
  ?character wdt:P509 ?cause .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)

 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.json_normalize(results['results']['bindings'])
results_df[['causeLabel.value', 'count.value']]

,causeLabel.value,count.value
0,gunshot wound,56
1,decapitation,55
2,killed in action,45
3,stroke,24
4,train wreck,22
...,...,...
81,falling from height,2
82,aviation accident,2
83,http://www.wikidata.org/.well-known/genid/0167...,2
84,lung disease,2



#### **Query 6**:  List 100 youngest authors alive from the Eastern Europe. ####

In [22]:
sparql.setQuery("""
SELECT DISTINCT ?personLabel ?countryLabel ?age
WHERE {
  ?person wdt:P106 wd:Q36180 ; #occupation writer
          wdt:P27 ?country ; #country of citizenship
          wdt:P569 ?birth_date.

  BIND(year(now()) - year(?birth_date) AS ?age)
  FILTER NOT EXISTS { ?person wdt:P570 ?death } #no death date for this person exist

  ?country wdt:P361 wd:Q27468 . #country is a part of Easter Europe

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en". }
 }

ORDER BY (?age)
LIMIT 100

 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.json_normalize(results['results']['bindings'])
results_df[['personLabel.value', 'countryLabel.value', 'age.value']]

,personLabel.value,countryLabel.value,age.value
0,Silva Souriau,Romania,NaN
1,Vojta Horák,Czech Republic,16
2,Magdaléna Paťhová,Czech Republic,16
3,Coolawik,Poland,18
4,Tomáš Grygar,Czech Republic,18
...,...,...,...
95,Nikita Laptinskiy,Belarus,31
96,Marie Donajová,Czech Republic,31
97,Kateřina Brabcová,Czech Republic,31
98,Ester Bezděk,Czech Republic,31
